# Playing with Color
stough 202-

In this activity we'll use what we've learned about color to try and **enhance** or even **compress** them without too much perceptual loss. The other notebooks in this folder should be useful here, particularly [color_intro](./color_intro.ipynb), [color_HSV](./color_HSV.ipynb), and [color_YCbCr](./color_YCbCr.ipynb) 

## Imports
You can borrow the imports from any of the other notebooks in this directory.

In [9]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import skimage.color as color

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils
from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists)

## 1. Enhancement: Improving the Color of an Image
Take a look at  `../dip_pics/sf.jpg` and try the `vis_hists` function on it to the distributions of the R,G,B channels. Of course you can try this out with any image you like of course.

In [10]:
I = plt.imread('../dip_pics/sf.jpg')

In [11]:
vis_hists(I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

To me it looks like the clouds shrouding the [Golden Gate Bridge](https://en.wikipedia.org/wiki/Golden_Gate_Bridge) are too dark, not blue enough really for my tastes. You can see this in the relative color distributions as well, with the blue component maxing out below the midpoint of the intensity range. 

Try adding some blue to the image to really make those clouds pop better. **Be careful** to use the `arr_info` to understand both the `dtype` and understood range of your data. 

In [12]:
J = I.copy()
J[...,2] = np.where(J[...,2]*1.5 > 255)
plt.figure(figsize=(4,4))
plt.imshow(I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
matrix_utils.arr_info(I)

((947, 1421, 3), dtype('uint8'), 0, 255)

## 2. Ehancement: Improving Contrast through Saturation of Color
Take a look at `../dip_pics/world_overexposed.jpg`. Here we have a very overexposed image. (Again, find any overexposed image you like). What this means generally is that we have a lot of what should be color/Hue showing up as mostly white; as in, it's been mixed with too much white. See the [color_HSV](./color_HSV.ipynb) demo titled "Viewing the planes of the HSV space" and see what happens as saturation $S_{HSV}$ approaches zero. 

Try converting the image to HSV and then scaling the saturation component to improve the color purity of all those washed-out pixels. Then convert back to RGB and see if it's improved. **Remember**, from the [skimage documentation](https://scikit-image.org/docs/dev/api/skimage.color.html#skimage.color.rgb2hsv), that the hsv channels are all in $[0,1]$, so when you do the scaling you might need to `np.clip`. 

In [13]:
K = plt.imread('../dip_pics/world_overexposed.jpg')
K = np.array(K)
plt.figure(figsize=(4,4))
plt.imshow(K)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
Khsv = color.rgb2hsv(K)
Khsv[...,1] += 0.3
K = color.hsv2rgb(Khsv)
plt.figure(figsize=(4,4))
plt.show(K)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 3. Compression: Reducing the Bit-Depth of an Image to Save Space
In a prior activity we reduced the bit-depth of the RGB color channels of an image and saw how that *quantization* affected the image quality, creating sharp edges of color change in what would be smoothly graded areas. So one of the limits of potentially compressing images relates to the minimum bit-depth that can maintain perceptual fidelity.

But in this module's materials, we've seen that some color spaces (HSV, YCbCr, and L*a*b*) include some dimension that captures the brightness/luminance/intensity representation of an image separately from the "color" aspect. [YCbCr](./color_YCbCr.ipynb) in particular is used a lot in compression for precisely this reason.

In the below cells, try considering an image like `../dip_pics/bellagio.jpg` or `../dip_pics/mountainSpring.jpg` for example. 

- See what happens with you reduce the bit-depth to 4-4-4 in RGB. 
- Then try reducing the bit-depth to 6-3-3 but **in the YCbCr** space. Here we're saving the same number of bits per pixel, but differentially quantizing (and therefore compressing) the **Luminance** and color components.

It may be useful here to define a function that quantizes a 2D `ndarray` instead of a whole image at once. Luckily, it's likely the same quantization function you've already written may suffice. I include a useable version here just to help.  

**Be careful**: [`skimage.color.ycbcr2rgb`](https://scikit-image.org/docs/dev/api/skimage.color.html#skimage.color.ycbcr2rgb) expects 'float64' data type. 

In [8]:
def quantize_bitdepth(C, bitdepth=8):
    """
    quantize_bitdepth(C, bitdepth=8): return a copy of the array C where the elements are
    limited to 2**bitdepth unique values. Tries to handle a float image just in case.
    """
    assert bitdepth <= 8, f'quantize_bitdepth error: expects bitdepth <= 8, got {bitdepth}'
    
    shft_amt = 8 - bitdepth
    
    J = np.uint8(C)
    if C.dtype == 'float':
        J = np.uint8(256 * (J/J.max()))
        
    return np.uint8((J >> shft_amt)*(255/(2**bitdepth-1)))